In [1]:
#Run this cell
#Importing necessary libraries 
import pandas as pd  
import numpy as np 
import math
import matplotlib.pyplot as plt
import json
ans=[0]*5

In [2]:
#Import the dataset and define the feature as well as the target datasets / columns 
df = pd.read_csv("zoo.csv") 
#We drop the animal names since this is not a good feature to split the data on. 
data = df.drop('animal_name', axis =1)
data

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [3]:
#Write a function to find the entropy on a split "target_col"
#def entropy(target_col):
def find_entropy (a):
    count = []
    entropy = 0
    for x in a.unique():
        mask = (a == x)
        counter = len(a.where(mask).dropna())
        prob = counter / len(a)
        count.append ((x, counter, prob))
        entropy += prob*(math.log(prob,2)) 
        #print (entropy)
    #print (count)
    return -1*entropy

In [4]:
#Find the entropy of all the features in the dataset
#Save all the feature names in an array "feature names"
feature_names=['hair','feathers','eggs','milk','airborne','aquatic','predator','toothed','backbone', 
               'breathes','venomous','fins','legs','tail','domestic','catsize']
entropies = []
for  x in feature_names:
    entropies.append((x, find_entropy(data[x])))
entropies

[('hair', 0.9840304711717017),
 ('feathers', 0.7179499765002912),
 ('eggs', 0.9794662187017299),
 ('milk', 0.9743197211096903),
 ('airborne', 0.7910662980902585),
 ('aquatic', 0.9396846718728563),
 ('predator', 0.9914266810680207),
 ('toothed', 0.9685867165455516),
 ('backbone', 0.6761627418829198),
 ('breathes', 0.7374895672137456),
 ('venomous', 0.3993820824245975),
 ('fins', 0.653839880626333),
 ('legs', 2.0338113440641234),
 ('tail', 0.8228368841492257),
 ('domestic', 0.5538976334852962),
 ('catsize', 0.9880162151534646)]

In [5]:
#Find the entropy of the feature "toothed"
ans[0]= find_entropy(data["toothed"])

In [6]:
#Write a function to calculate Information Gain on a split attribute and a target column
def InfoGain(data,split_attribute_name,target_name="class_type"):       
    #Calculate the entropy of the total dataset  
    total_entropy = find_entropy(data[target_name])
    #Calculate the values and the corresponding counts for the split attribute   
    count = []
    entropy = 0
    Weighted_Entropy =0
    for x in data[split_attribute_name].unique():
        mask = (data[split_attribute_name] == x)
        counter = len(data[split_attribute_name].where(mask).dropna())
        prob = counter / len(data[split_attribute_name])
        count.append ((x, counter, prob))
    #Calculate the weighted entropy
    for i in range(len(count)):
        mask = (data[split_attribute_name]==count[i][0])
        a = data.where(mask).dropna()[target_name]
        Weighted_Entropy = Weighted_Entropy + count[i][2]*find_entropy(a) 
    #Calculate the information gain  
    Information_Gain = total_entropy - Weighted_Entropy
    #print(np.sum(counts))
    return Information_Gain

In [7]:
ans[1]=InfoGain(data,'hair',"milk")

In [8]:
#Find the Info gain having "milk" as the split attribute and all the other features as target features one at a time
InfoGain(data,'milk')

0.9743197211096903

In [9]:
#Import Decision Tree Classifier from sklearn 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
X = data.drop('class_type', axis =1)
y = df['class_type']
#Split the given data into 80 percent training data and 20 percent testing data
X_train,X_test, y_train, y_test=train_test_split(X, y,train_size=0.8,random_state=1)

In [10]:
#Fit the given data
decision_tree = DecisionTreeClassifier(random_state=42)   
decision_tree.fit(X_train,y_train)
pred = decision_tree.predict(X_test)

In [11]:
#Make a prediction on the test data and return the percentage of accuracy
ans[2]= accuracy_score(y_test, pred)

In [12]:
#Run this cell to visualize the decision tree
from six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()
export_graphviz(decision_tree, out_file=dot_data, feature_names=feature_names,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

ModuleNotFoundError: No module named 'pydotplus'

In [18]:
#Use sklearn to make a classification report and a confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred, zero_division=0))
result = precision_recall_fscore_support(y_test, pred)

[[11  0  0  0  0  0]
 [ 0  5  0  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0  0  0  0  2]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00         5
           3       0.50      1.00      0.67         1
           4       1.00      1.00      1.00         1
           5       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         2

    accuracy                           0.95        21
   macro avg       0.75      0.83      0.78        21
weighted avg       0.93      0.95      0.94        21



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
#Find the recall,f1-score for class type '3'
ans[3]= [result[2][1],result[2][2]]

In [20]:
#Calculate Mean Absolute Error,Mean Squared Error and Root Mean Squared Error
from sklearn.metrics import mean_absolute_error
mse=metrics.mean_squared_error(y_test,pred)
rmse=math.sqrt(mse)
mae=metrics.mean_absolute_error(y_test,pred)

In [21]:
#Find the mean absolute error and root mean square error, save then in a list [mae,rmse]
ans[4]=[mae,rmse]

In [22]:
##do not change this code
import json
ans = [str(item) for item in ans]

filename = ""

# Eg if your name is Saurav Joshi and email id is sauravjoshi123@gmail.com, filename becomes
# filename = sauravjoshi123@gmail.com_Saurav_Joshi_LinearRegression

In [23]:
ans

['0.9685867165455516',
 '0.6599660577558697',
 '0.9523809523809523',
 '[1.0, 0.6666666666666666]',
 '[0.09523809523809523, 0.4364357804719847]']

## Do not change anything below!!
- Make sure you have changed the above variable "filename" with the correct value. Do not change anything below!!

In [ ]:
from importlib import import_module
import os
from pprint import pprint

findScore = import_module('findScore')
response = findScore.main(ans)
response['details'] = filename
with open(f'evaluation_{filename}.json', 'w') as outfile:
    json.dump(response, outfile)
pprint(response)